In [170]:
import sys
import os
import time
import pandas as pd
import numpy as np
import collections

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.metrics import classification_report
#from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import StratifiedKFold
#from sklearn.model_selection import RepeatedStratifiedKFold

In [181]:
iterations = 3
test_size = 0.8
random_state = 0


In [182]:
def classifaction_report_dataFrame(report):
    report_data = []
    lines = report.split('\n')
    for line in lines[2:-3]:
        row = {}
        row_data = line.split('      ')
        row['class'] = row_data[0]
        row['precision'] = float(row_data[1])
        row['recall'] = float(row_data[2])
        row['f1_score'] = float(row_data[3])
        row['support'] = float(row_data[4])
        report_data.append(row)
    return report_data

In [183]:
data = np.empty((0))
data_labels = np.empty((0))

precision_positive = 0
recall_positive = 0
f1score_positive = 0

precision_negative = 0
recall_negative = 0
f1score_negative = 0


comments = pd.read_csv('FilmWeb-commentsRates.csv')
comments = comments.reset_index()
comments.loc[comments.emotion == 'neutral', 'emotion'] = "negative"
comments.groupby(['emotion']).size()
for index, comment in comments.iterrows():
    data = np.append(data, comment['Text'])
    data_labels = np.append(data_labels, comment['emotion'])


print 'number of dataset: ' + str(iterations)
report_data_Linear = pd.DataFrame()
report_data_Liblinear = pd.DataFrame()

#Some classification problems can exhibit a large imbalance in the distribution of the target classes: 
#for instance there could be several times more negative samples than positive samples.
#StratifiedKFold is a variation of k-fold which returns stratified folds: each set contains approximately 
#the same percentage of samples of each target class as the complete set.
#K-krotna walidacja
sss = StratifiedKFold(n_splits=iterations)
for train_index, test_index in sss.split(data, data_labels):
    train_data, test_data = data[train_index], data[test_index]
    train_labels, test_labels = data_labels[train_index], data_labels[test_index]

    # Create feature vectors
    vectorizer = TfidfVectorizer(min_df=5,max_df = 0.8,sublinear_tf=True,use_idf=True)
    train_vectors = vectorizer.fit_transform(train_data)
    test_vectors = vectorizer.transform(test_data)

    # Perform classification with SVM, kernel=linear
    classifier_linear = svm.SVC(kernel='linear')
    t0 = time.time()
    classifier_linear.fit(train_vectors, train_labels)
    t1 = time.time()
    prediction_linear = classifier_linear.predict(test_vectors)
    t2 = time.time()
    time_linear_train = t1-t0
    time_linear_predict = t2-t1

    # Perform classification with SVM, kernel=linear
    classifier_liblinear = svm.LinearSVC()
    t0 = time.time()
    classifier_liblinear.fit(train_vectors, train_labels)
    t1 = time.time()
    prediction_liblinear = classifier_liblinear.predict(test_vectors)
    t2 = time.time()
    time_liblinear_train = t1-t0
    time_liblinear_predict = t2-t1

    reportLinear = classification_report(test_labels, prediction_linear)
    reportLiblinear = classification_report(test_labels, prediction_liblinear)

    classifications_report_Linear = classifaction_report_dataFrame(reportLinear)
    report_data_Linear.append(classifications_report_Linear)
    classifications_report_Liblinear = classifaction_report_dataFrame(reportLiblinear)
    report_data_Liblinear.append(classifications_report_Liblinear)
    
print 'Training data:'
counter=collections.Counter(train_labels)
print(counter)

print 'Test data:'
counter=collections.Counter(test_labels)
print(counter)

number of dataset: 3
Training data:
Counter({'positive': 2047, 'negative': 924})
Test data:
Counter({'positive': 1023, 'negative': 461})


In [184]:
report_data_Linear

""


In [67]:
# Print results in a nice table
    print("Results for SVC(kernel=linear)")
    print("Training time: %fs; Prediction time: %fs" % (time_linear_train, time_linear_predict))
    print(classification_report(test_labels, prediction_linear))
    print("Results for LinearSVC()")
    print("Training time: %fs; Prediction time: %fs" % (time_liblinear_train, time_liblinear_predict))
    print(classification_report(test_labels, prediction_liblinear))

Results for SVC(kernel=rbf)
Training time: 2.440635s; Prediction time: 0.266033s
             precision    recall  f1-score   support

   negative       0.00      0.00      0.00       171
   positive       0.62      1.00      0.76       274

avg / total       0.38      0.62      0.47       445

